In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import json
import time

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Path to your ChromeDriver
service = Service('/path/to/chromedriver')  # Update this path

# Initialize WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Function to scrape posts from a single page
def scrape_posts(page_number):
    url = f'https://georgerrmartin.com/notablog/page/{page_number}/'
    driver.get(url)
    time.sleep(3)  # Wait for the page to load

    posts = []
    
    post_elements = driver.find_elements(By.CSS_SELECTOR, 'article.post')

    for post in post_elements:
        # Extract the title, date, and content of the post
        title = post.find_element(By.CSS_SELECTOR, 'h2.entry-title').text.strip()
        date = post.find_element(By.CSS_SELECTOR, 'time.entry-date').text.strip()
        content = post.find_element(By.CSS_SELECTOR, 'div.entry-content').text.strip()

        # Add the post data to the list
        posts.append({
            'title': title,
            'date': date,
            'content': content
        })

    return posts

# Function to scrape all posts from all pages
def scrape_all_posts():
    all_posts = []
    page_number = 1

    while True:
        print(f'Scraping page {page_number}...')
        posts = scrape_posts(page_number)
        if not posts:
            break
        all_posts.extend(posts)
        page_number += 1

    return all_posts

# Run the scraper and collect all posts
all_posts = scrape_all_posts()

# Print the number of posts collected
print(f'Total posts collected: {len(all_posts)}')

# Optionally, save the posts to a file (e.g., JSON or CSV)
with open('grrm_notablog_posts.json', 'w') as f:
    json.dump(all_posts, f, indent=4)

print('Posts saved to grrm_notablog_posts.json')

# Close the WebDriver
driver.quit()


Scraping page 1...
Total posts collected: 0
Posts saved to grrm_notablog_posts.json
